In [1]:

!pip install ultralytics opencv-python


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
from ultralytics import YOLO
import cv2
import time
import json
import os

In [9]:
def AlertLog(timestamp, conf, cameraID,bbox):

    Alert = {
        "confidence": round(float(conf),3),
        "timestamp": timestamp,
        "cameraID": cameraID,
        "bbox": bbox

            }
    with open("alerts.json", "w") as f:
        f.write(json.dumps(Alert) + "\n")

        print("Alert logged: ", Alert)



In [10]:
yaml_content = """
train: data/train/images
val:   data/valid/images
test:  data/test/images


nc: 1
names: ['fire']
"""

with open("fire1.yaml", "w") as f:
    f.write(yaml_content)

In [11]:

model = YOLO("yolov8n.pt")
results = model.train(
    data = "fire1.yaml",
    epochs = 100,
    imgsz = 640,
    batch = 8
)


New https://pypi.org/project/ultralytics/8.3.201 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.196  Python-3.12.10 torch-2.8.0+cpu CPU (Intel Core Ultra 7 155U)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=fire1.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train4, nbs=64, nms=False, opset=None, optimize=False, op

In [20]:

# 25 epochs of 16 image batches took exactly 10 minutes. /train22 without preloading yolov8n.pt
# 2nd attempt. 25 epochs of 16 image batches preloaded with yolov8n.pt took exactly 30+ minutes and bugged on epoch 23
#3rd attempt. same as above. Took exactly 10-11 minutes. /train3
#4th attempt due to some bugs, same as above. saved to train 6
#5th attempt. previous attempts gave bad confidence. training 100 epochs on train 6 and not yolo model. should give better results. took 43 mins. saved to /train7
# 6th attempt. just for good measure. previous was also pretty bad. lr = 0.003. trained on train7. 200 epochs.should take 1.5hrs. took exactly 1hr 38 mins. saved to /train16.not good still.
# 7th attempt. new dataset. bigger. doing the labels myself. 25 epochs. took 54 mins. not bad. saved to /train20. didnt work.
# final attempt, biggest dataset, normal method. thousands of images.100 epochs. should take 24 hours lol.

In [14]:
model = YOLO("runs/detect/train4/weights/best.pt")

In [6]:
import torch
import torchvision.transforms as transforms
from PIL import Image

In [1]:
import cv2, numpy as np, time, json
from ultralytics import YOLO

sources = [
    "CCTV1.mp4","CCTV2.mp4","CCTV3.mp4",
    "CCTV4.mp4","CCTV5.mp4","CCTV6.mp4", "CCTV7.mp4","CCTV8.mp4",
    "CCTV9.mp4", "CCTV10.mp4"
]
caps = [cv2.VideoCapture(src) for src in sources]
model = YOLO("runs/detect/train4/weights/best.pt")

frame_skip = 1
frame_count = 0
target_size = (320, 240)  # bigger size → better quality

cv2.namedWindow("Control Room", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Control Room", 1280, 720)

def AlertLog(cam_id, cls, conf, bbox, ts):
    alert = {"camera":cam_id+1,"class":cls,"conf":round(conf,3),
             "bbox":bbox,"timestamp":ts}
    with open("alerts.jsonl","a") as f: f.write(json.dumps(alert)+"\n")
    print("ALERT:", alert)

while True:
    frames = []
    frame_count += 1

    for cam_id, cap in enumerate(caps):
        ret, frame = cap.read()

        # loop video if ended
        if not ret or frame is None:
            cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
            ret, frame = cap.read()

        if not ret or frame is None:
            # offline placeholder
            frame = np.zeros((target_size[1], target_size[0], 3), np.uint8)
            cv2.putText(frame, f"Camera {cam_id +1} Offline", (30, 120),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
        else:
            frame = cv2.resize(frame, target_size)
            cv2.putText(frame, f"Cam {cam_id +1 }", (5, 20),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 2)

            if frame_count % frame_skip == 0:
                results = model.predict(frame, conf=0.25, verbose=False)
                for r in results:
                    for box in r.boxes:
                        conf = float(box.conf[0])
                        if conf >= 0.80:
                            x1, y1, x2, y2 = map(int, box.xyxy[0])
                            cls_id = int(box.cls[0])
                            cls = r.names[cls_id]
                            ts = time.strftime("%Y-%m-%d %H:%M:%S")
                            AlertLog(cam_id, cls, conf, [x1,y1,x2,y2], ts)
                            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 1)
                            cv2.putText(frame, f"{cls} {conf:.2f}",
                                        (x1, max(0, y1 - 5)),
                                        cv2.FONT_HERSHEY_SIMPLEX, 0.4,
                                        (0, 0, 255), 1)

        frames.append(frame)

    # stack grid
    row1 = np.hstack(frames[:5])
    row2 = np.hstack(frames[5:])
    grid = np.vstack([row1, row2])

    # scale to fit screen
    display_grid = cv2.resize(grid, (1280, 720), interpolation=cv2.INTER_LINEAR)

    cv2.imshow("Control Room", display_grid)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    time.sleep(0.01)

for cap in caps: cap.release()
cv2.destroyAllWindows()


ALERT: {'camera': 8, 'class': 'fire', 'conf': 0.824, 'bbox': [95, 76, 186, 116], 'timestamp': '2025-09-21 11:07:29'}
ALERT: {'camera': 9, 'class': 'fire', 'conf': 0.802, 'bbox': [147, 32, 240, 121], 'timestamp': '2025-09-21 11:07:29'}
ALERT: {'camera': 8, 'class': 'fire', 'conf': 0.828, 'bbox': [96, 74, 185, 116], 'timestamp': '2025-09-21 11:07:30'}
ALERT: {'camera': 1, 'class': 'fire', 'conf': 0.803, 'bbox': [142, 106, 181, 164], 'timestamp': '2025-09-21 11:07:31'}
ALERT: {'camera': 8, 'class': 'fire', 'conf': 0.838, 'bbox': [95, 73, 186, 117], 'timestamp': '2025-09-21 11:07:31'}
ALERT: {'camera': 8, 'class': 'fire', 'conf': 0.827, 'bbox': [95, 73, 186, 118], 'timestamp': '2025-09-21 11:07:32'}
ALERT: {'camera': 8, 'class': 'fire', 'conf': 0.818, 'bbox': [96, 76, 186, 118], 'timestamp': '2025-09-21 11:07:33'}
ALERT: {'camera': 8, 'class': 'fire', 'conf': 0.806, 'bbox': [98, 77, 185, 117], 'timestamp': '2025-09-21 11:07:34'}
ALERT: {'camera': 8, 'class': 'fire', 'conf': 0.81, 'bbox': [

In [6]:
model.export(format="onnx")


Ultralytics 8.3.196  Python-3.12.10 torch-2.8.0+cpu CPU (Intel Core Ultra 7 155U)

PyTorch: starting from 'runs\detect\train4\weights\best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 5, 8400) (6.0 MB)

ONNX: starting export with onnx 1.19.0 opset 19...
ONNX: slimming with onnxslim 0.1.69...
ONNX: export success  0.6s, saved as 'runs\detect\train4\weights\best.onnx' (11.7 MB)

Export complete (0.7s)
Results saved to C:\Users\Eng.Omran\FirePrototype\runs\detect\train4\weights
Predict:         yolo predict task=detect model=runs\detect\train4\weights\best.onnx imgsz=640  
Validate:        yolo val task=detect model=runs\detect\train4\weights\best.onnx imgsz=640 data=fire1.yaml  
Visualize:       https://netron.app


'runs\\detect\\train4\\weights\\best.onnx'

In [20]:
import onnxruntime as ort

session = ort.InferenceSession("runs/detect/train4/weights/best.onnx")

# 2) Define one video source (replace with RTSP URL or MP4)
source = "CCTV8.mp4"
cap = cv2.VideoCapture(source)

cv2.namedWindow("ONNX Camera", cv2.WINDOW_NORMAL)  # make window resizable
cv2.resizeWindow("ONNX Camera", 640, 480)

# 3) Preprocessing function
def preprocess(frame, img_size=640):
    # Resize to model size (640x640)
    img = cv2.resize(frame, (img_size, img_size))
    # Convert HWC → CHW (channels first)
    img = img.transpose((2, 0, 1))
    # Normalize (0-1 range)
    img = img / 255.0
    # Add batch dimension
    img = np.expand_dims(img, axis=0).astype(np.float32)
    return img

# 4) Main loop
while True:
    ret, frame = cap.read()
    if not ret:
        print("Camera offline or video ended")
        break

    # Preprocess frame
    img = preprocess(frame)

    # Run inference
    outputs = session.run(None, {"images": img})

    # TODO: Parse YOLO outputs into boxes, scores, classes
    # For now, just show "Processed"
    cv2.putText(frame, "Processed", (30, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)

    # Show video
    cv2.imshow("ONNX Camera", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cleanup
cap.release()
cv2.destroyAllWindows()
